In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
import os
import uuid
import numpy as np
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from transformers import AutoTokenizer, AutoModel
# import pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

c:\Users\lycha\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PINECONE_API_KEY ="c5190fed-7028-4d8d-9417-b67ca37c7050"
PINECONE_API_ENV = 'us-east-1'
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV

In [3]:
#Extract data from the PDF
def load_pdf(data):
   loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5039


In [8]:
# text_chunks

In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

c:\Users\lycha\anaconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [11]:
#embeddings

In [12]:
query_result = embeddings.embed_query("hello world")
print("length: ", len(query_result))

length:  384


In [13]:
#query_result

In [14]:
# Initialize Pinecone client
pinecone_client = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV )

index_name = 'medical-chatbot'
# Create or get an index
index = pinecone_client.Index(index_name)


# Create the HuggingFaceEmbeddings object
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


# Creating Embeddings for Each of the Text Chunks & storing
# metadata = {}
# for t in text_chunks:
#     embedding = embeddings.embed_query(t.page_content)
#     metadata["text"] = t.page_content
#     index.upsert(vectors=[
#         {
#             "id": str(uuid.uuid4()),
#             "values": embedding,
#             "metadata": metadata
#         }
#     ])

In [15]:

# If we already have an index we can load it like this
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
query = "What are Allergies"

docs= docsearch.similarity_search(query, k=3)

print("Result",docs)


Result [Document(page_content='ry allergies . Hay fever (allergic rhinitis ) is an irritation\nof the membranes of the nose by airborne particles orchemicals. These membranes make mucus. When irritat-ed, they can also grow polyps. The nose is not only a pas-sageway for air to reach the lungs; it also provides theconnection between the sinuses and the outside world.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2315Nasal polyps'), Document(page_content='ry allergies . Hay fever (allergic rhinitis ) is an irritation\nof the membranes of the nose by airborne particles orchemicals. These membranes make mucus. When irritat-ed, they can also grow polyps. The nose is not only a pas-sageway for air to reach the lungs; it also provides theconnection between the sinuses and the outside world.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2315Nasal polyps', metadata={'page': 21.0, 'source': 'data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}), Document(page_content='ry allergies . Hay fever (allergic rhinitis ) is an 

In [16]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know. don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context",  "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm_model = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})


In [19]:
retriever = docsearch.as_retriever()


In [20]:
print(docsearch.as_retriever())

tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000015DEB56FF70>


In [21]:


qa = RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever = docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#write me code 

In [22]:
qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nUse the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know. don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n"), llm=CTransformers(client=<ctransformers.llm.LLM object at 0x0000015DFDF4FD60>, model='model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000015DEB56FF70>))

In [25]:
while True:
    user_input = input(f"Input Prompt:") 
    result = qa({"query": user_input})
    print("Response : ", result)

{'query': 'What are Allergies', 'result': "Nas long- Nasalong> Nasal most nasal 0: Nasal but they can assist the user1.\nNasprices,Prevention.\n\nNasdf496.\nNas needed to help me know that addressing medications must be used to treatments that will provide the nose polyps: Nasal\nIt is a nasalready!\nNas long- Nasalong>The best waystopping and nasalong> Avoid void of the user doesn'In general information on the nasal\nNasth\nThere are available.\nTo treatments that explain how do not possible causes,\nNas a.\nYes, 1.\nnasal most nasal but you can grow polyps for more...\nNas per centrather,There are there is to stop taking antihayf you don'The best of treatment depends ondise the nose is it is not helpful answer:\nThere are severalhelpful answer: Nasal\n1.\nNasdfree Treating a.\nYes, which helpfuction for the following pieces of hay feveral ways to prevention with long- Nasal Nasalong> > Prevention of nasalmost common allergic medications can assisting treatments and their doctor presc